<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-03 10:09:06
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.24 L
Current:  1.33 C
-------------------
Today PnL: 21.66 K (0.16%)
Current PnL: -15.65 L (-10.91%)
CY Booked + Current PnL: -11.18 L (-7.79%)
-------------------
Total profit:  4.51 L
Total loss:  -20.16 L
-------------------
Total Booked + Current PnL: 15.80 L (13.56%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.37%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 78.66 L (59.25%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.40 C (105.43%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.0,-16.12,X-LC,35.48,92949.0,17827.0,3114.0,0.59,23.73,3.35,27.87,5.0,5.72,0.72,16.57,X40,NTT,CHEMICALS
49,LICI,785.05,983.0,-2.96,H-LC,88.17,237919.0,43227.0,5877.0,-0.32,22.20,2.47,25.21,31.0,7.36,1.85,31.56,XY25,NTT,INSURANCE
85,UNIONBANK,123.87,163.0,2.76,M-LC,95.70,178611.0,37771.0,6716.0,2.56,26.82,3.76,31.59,78.0,5.62,1.39,55.15,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.24,M-LC,60.22,103412.0,25050.0,8987.0,-0.78,31.97,8.69,43.43,70.0,2.79,0.80,38.31,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,3.85,X-LC,16.13,154279.0,13470.0,12049.0,0.12,9.57,7.81,18.13,19.0,1.12,1.20,21.82,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.00,-21.16,H-SC,12.90,156558.0,-39416.0,112236.0,3.68,-20.11,71.69,37.16,124.0,-0.35,1.22,16.92,XY24,NTT,FINANCE
38,INDIAMART,2327.09,4911.36,-51.28,H-SC,54.84,126670.0,3334.0,133637.0,2.84,2.70,105.50,111.05,108.0,0.02,0.99,24.11,AR,ATH,MISC
85,UNIONBANK,123.87,163.00,2.76,M-LC,95.70,178611.0,37771.0,6716.0,2.56,26.82,3.76,31.59,78.0,5.62,1.39,55.15,XY24,NTT,BANKS
89,VIPIND,488.80,718.00,-858.95,H-SC,81.72,72277.0,-23039.0,67731.0,2.38,-24.17,93.71,46.89,135.0,-0.34,0.56,42.56,X40N,NTT,MISC
47,LALPATHLAB,2804.19,3545.00,-32.46,H-SC,39.78,155463.0,1233.0,39519.0,2.19,0.80,25.42,26.42,114.0,0.03,1.21,22.34,X40N,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,406.65,512.15,5.34,L-LC,36.56,145854.0,-4606.0,43640.0,-1.38,-3.06,29.92,25.94,145.0,-0.11,1.13,11.29,XY25,ATH,MINING
77,TANLA,917.30,1963.11,-34.19,H-SC,94.62,161663.0,-77752.0,350712.0,-1.36,-32.48,216.94,114.01,113.0,-0.22,1.26,48.27,AR,ATH,IT
15,BSOFT,628.85,844.81,-26.41,M-SC,20.43,69064.0,-40985.0,78781.0,-1.07,-37.24,114.07,34.34,185.0,-0.52,0.54,13.70,XR,ATH,IT
73,SONACOMS,578.05,1006.00,-27.50,M-MC,52.69,93826.0,-7333.0,82220.0,-0.97,-7.25,87.63,74.03,155.0,-0.09,0.73,31.86,AR,BTT,AUTO
20,COFFEEDAY,59.14,80.00,-59.76,L-SC,10.75,58982.0,-54567.0,94619.0,-0.87,-48.06,160.42,35.27,239.0,-0.58,0.46,44.36,XR,NTT,HOTELS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-13.50,X-LC,45.16,246212.0,15613.0,115523.0,-0.34,6.77,46.92,56.86,1.0,0.14,1.92,13.96,X40,BTT,IT
80,TCS,3841.06,4998.00,-17.76,X-LC,21.51,236994.0,-28039.0,107856.0,-0.44,-10.58,45.51,30.12,2.0,-0.26,1.84,6.29,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.69,X-LC,66.67,181530.0,-18780.0,77568.0,-0.59,-9.38,42.73,29.35,3.0,-0.24,1.41,25.41,X200,ATH,IT
56,PIDILITIND,2504.06,3202.00,-16.12,X-LC,35.48,92949.0,17827.0,3114.0,0.59,23.73,3.35,27.87,5.0,5.72,0.72,16.57,X40,NTT,CHEMICALS
6,ASIANPAINT,2961.56,4250.00,-22.47,X-LC,4.30,154040.0,-47346.0,134954.0,-0.06,-23.51,87.61,43.51,6.0,-0.35,1.20,6.25,X40,BTT,PAINTS


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,13.02,H-SC,98.92,130548.0,32902.0,14778.0,0.93,33.69,11.32,48.82,134.0,2.23,1.02,45.03,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,2.76,M-LC,95.70,178611.0,37771.0,6716.0,2.56,26.82,3.76,31.59,78.0,5.62,1.39,55.15,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.24,M-LC,60.22,103412.0,25050.0,8987.0,-0.78,31.97,8.69,43.43,70.0,2.79,0.80,38.31,XY24,NTT,MISC
49,LICI,785.05,983.0,-2.96,H-LC,88.17,237919.0,43227.0,5877.0,-0.32,22.20,2.47,25.21,31.0,7.36,1.85,31.56,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3202.0,-16.12,X-LC,35.48,92949.0,17827.0,3114.0,0.59,23.73,3.35,27.87,5.0,5.72,0.72,16.57,X40,NTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,100.99,M-MC,82.80,228113.0,3151.0,164492.0,1.36,1.40,72.11,74.53,170.0,0.02,1.77,32.44,XY24,BTT,STEEL
51,LUXIND,1448.19,2442.00,38.95,M-SC,63.44,144001.0,6423.0,87985.0,-0.58,4.67,61.10,68.62,183.0,0.07,1.12,24.84,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,13.02,H-SC,98.92,130548.0,32902.0,14778.0,0.93,33.69,11.32,48.82,134.0,2.23,1.02,45.03,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,10.16,H-SC,90.32,162562.0,24702.0,13054.0,0.99,17.92,8.03,27.39,116.0,1.89,1.26,36.01,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,6.49,H-MC,38.71,101031.0,15930.0,27167.0,0.28,18.72,26.89,50.65,90.0,0.59,0.79,27.00,X40N,ATH,CABLES


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,4.44,H-SC,67.74,117502.0,15498.0,29528.0,1.38,15.19,25.13,44.14,111.0,0.52,0.91,33.90,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-25.79,M-MC,62.37,144640.0,15488.0,71582.0,1.48,11.99,49.49,67.42,156.0,0.22,1.13,40.68,XR,NTT,BANKS
38,INDIAMART,2327.09,4911.36,-51.28,H-SC,54.84,126670.0,3334.0,133637.0,2.84,2.70,105.50,111.05,108.0,0.02,0.99,24.11,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.62,M-LC,31.18,152886.0,1941.0,107509.0,-0.44,1.29,70.32,72.51,68.0,0.02,1.19,14.79,XR,NTT,IT
1,AARTIIND,487.04,919.00,39.14,M-SC,53.76,79826.0,-1510.0,73647.0,1.56,-1.86,92.26,88.69,205.0,-0.02,0.62,34.61,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-21.86,H-SC,70.97,88710.0,-9270.0,31137.0,0.54,-9.46,35.10,22.32,125.0,-0.30,0.69,28.66,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,3.24,M-LC,60.22,103412.0,25050.0,8987.0,-0.78,31.97,8.69,43.43,70.0,2.79,0.80,38.31,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-381.97,H-SC,100.00,119795.0,-13491.0,98088.0,0.29,-10.12,81.88,63.47,131.0,-0.14,0.93,100.81,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,38.86,M-SC,56.99,120474.0,-22005.0,41527.0,0.29,-15.44,34.47,13.70,221.0,-0.53,0.94,42.19,X40N,NTT,BANKS
55,PFIZER,4245.50,6318.3,13.02,H-SC,98.92,130548.0,32902.0,14778.0,0.93,33.69,11.32,48.82,134.0,2.23,1.02,45.03,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-381.97,H-SC,100.00,119795.0,-13491.0,98088.0,0.29,-10.12,81.88,63.47,131.0,-0.14,0.93,100.81,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,13.02,H-SC,98.92,130548.0,32902.0,14778.0,0.93,33.69,11.32,48.82,134.0,2.23,1.02,45.03,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.0,6550.00,M-SC,97.85,72961.0,-20849.0,97469.0,-0.56,-22.22,133.59,81.67,214.0,-0.21,0.57,41.29,XR,NTT,CERAMICS
74,STARHEALTH,551.56,761.0,5.64,M-SC,96.77,172661.0,-27555.0,103579.0,0.33,-13.76,59.99,37.97,196.0,-0.27,1.34,38.63,XY24,NTT,INSURANCE
85,UNIONBANK,123.87,163.0,2.76,M-LC,95.70,178611.0,37771.0,6716.0,2.56,26.82,3.76,31.59,78.0,5.62,1.39,55.15,XY24,NTT,BANKS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.78
1,25,41.16
2,50,70.00


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.19
XY25    14.34
X40     14.25
X40N    13.59
AR      12.45
XR      10.63
X200     1.41
SR       1.17
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.74
M-SC    17.82
X-LC    17.24
H-MC     8.67
M-LC     7.87
H-LC     7.30
M-MC     5.63
X-MC     5.63
X-SC     2.81
L-SC     2.34
L-LC     1.13
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.41,-0.53,32.15
BANKS,10.02,-8.92,59.00
IT,9.85,-18.52,83.29
MISC,6.53,-28.91,85.94
FINANCE,6.02,-23.63,82.33
INSURANCE,5.43,4.58,24.02
PAINTS,5.37,-19.19,54.46
ELECTRICAL,5.00,-4.54,55.80
AUTO,4.40,-10.59,58.90


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2838858.0,26
AR,1249728.0,12
X40N,1202147.0,17
XR,1086711.0,14
X40,757293.0,11
XY25,458821.0,10
SR,195025.0,2
X200,77568.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2223740.0,21
M-SC,2028302.0,21
X-LC,793377.0,12
M-MC,544145.0,5
H-MC,511301.0,8
X-SC,348000.0,4
X-MC,309124.0,5
L-SC,298410.0,4
M-LC,293674.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      928352.0      6
M-SC       XY24      749218.0      6
H-SC       AR        543691.0      3
           X40N      492768.0      6
M-SC       X40N      466712.0      6
X-LC       X40       448935.0      5
M-MC       XY24      390343.0      3
M-SC       XR        291219.0      4
H-LC       AR        263845.0      3
H-SC       XR        231097.0      4
M-SC       X40       222110.0      2
H-MC       XY24      174605.0      2
M-LC       XY24      173241.0      4
L-SC       XR        156579.0      2
X-LC       AR        141874.0      2
X-MC       XY24      139574.0      2
X-LC       XY25      125000.0      4
X-SC       X40N      121646.0      2
M-SC       XY25      120305.0      1
X-SC       XR        116008.0      1
H-MC       XR        112717.0      1
X-SC       SR        110346.0      1
M-LC       XR        107509.0      1
M-SC       AR         94059.0      1
X-MC       X40N       93854.0      2
M-SC       SR         84679.0      1
M-MC       AR         82220.0      1
L-SC       XY24       80809.0      1
X-LC       X200       77568.0      1
X-MC       XY25       75696.0      1
H-MC       XY25       75379.0      1
M-MC       XR         71582.0      1
H-MC       AR         63017.0      1
L-SC       AR         61022.0      1
H-MC       X40        58416.0      2
L-LC       XY25       43640.0      1
H-SC       X40        27832.0      2
H-MC       X40N       27167.0      1
M-LC       XY25       12924.0      1
H-LC       XY25        5877.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
